# Cvičení 3

Tématem tohoto cvičení jsou přímé metody řešení soustav lineárních rovnic. Budeme se zabývat dopřednou a zpětnou substitucí a LU rozkladem.

## Dopředná a zpětná substituce

Základní princip přímých metod řešení soustav lineárních rovnic je převod matice soustavy na součin trojúhelníkových matic, např. 
$$\mathsf{A} = \mathsf{L}\mathsf{U}.$$
Soustavu $\mathsf{A}\mathbf{x} = \mathbf{b}$ tak převedeme na tvar
$$\mathsf{L}\mathsf{U}\mathbf{x} = \mathbf{b}$$
a vyřešíme ji ve dvou krocích 
1) $\mathsf{L}\mathbf{y}=\mathbf{b}$
2) $\mathsf{U}\mathbf{x}=\mathbf{y}$
pomocí dopředné a zpětné substituce.

V této části cvičení tedy nejdříve implementujeme funkce pro dopřednou a zpětnou substituci, které později využijeme k řešení soustav.

### Testovací matice

Nejprve vygenerujte testovací matice, pomocí kterých později zjistíte, zda vaše funkce pracují správně. Budete k tomu potřebovat
- funkci z knihovny NumPy, pomocí které vygenerujete náhodnou matici o daném rozměru (znáte z prvního cvičení),
- funkce `triu`, `tril` z knihovny NumPy (zjistěte v dokumentaci nebo pomocí Googlu, k čemu slouží a jak se používají)

In [3]:
# ÚKOL: importujte potřebnou knihovnu NumPy pod zkratkou np
import numpy as np

In [24]:
# ÚKOL: vygenerujte matici A náhodných čísel o rozměrech 4x4
A = np.random.rand(4, 4)

np.set_printoptions(precision=3) # pro lepší přehlednost
print(A)

[[0.339 0.888 0.37  0.358]
 [0.601 0.582 0.583 0.186]
 [0.339 0.642 0.105 0.936]
 [0.256 0.95  0.361 0.864]]


In [5]:
# ÚKOL: pomocí vhodných funkcí z knihovny NumPy extrahujte z matice A její horní a dolní trojúhelníkovou část.
U = np.triu(A)
print(U)
print("---------------------")

L = np.tril(A)
print(L)
print("---------------------")

# Ověřte funkčnost porovnáním s příslušnou částí matice A
B = L @ U
print(B)

[[0.314 0.622 0.068 0.201]
 [0.    0.53  0.691 0.695]
 [0.    0.    0.935 0.499]
 [0.    0.    0.    0.508]]
---------------------
[[0.314 0.    0.    0.   ]
 [0.301 0.53  0.    0.   ]
 [0.965 0.852 0.935 0.   ]
 [0.278 0.181 0.186 0.508]]
---------------------
[[0.099 0.195 0.021 0.063]
 [0.095 0.467 0.386 0.429]
 [0.303 1.052 1.529 1.254]
 [0.087 0.269 0.318 0.532]]


In [6]:
# ÚKOL: Vytvořte náhodný vektor b délky 4
b = np.random.rand(4)

print(b)

[0.206 0.219 0.8   0.232]


### Dopředná substituce

Soustavu s dolní trojúhelníkovou maticí $\mathsf{L}\mathbf{x}=\mathbf{b}$ můžeme efektivně řešit pomocí algoritmu dopředné substituce. 

1) Určíme první prvek vektoru řešení
$$x_1 = b_1 / L_{1, 1}$$

2) Zbývající prvky určíme jako 
$$\forall i \in \{2, 3, \ldots, m\}: x_i = (b_i - \sum_{j=1}^{i-1}L_{i, j} x_j) / L_{i, i}$$

Na základě těchto předpisů doplňte chybějící části následujícího kódu. Nezapomeňte ale, že v Pythonu indexujeme od nuly.

In [7]:
# ÚKOL: Doplňte následující kód

def fsubst(L, b):
    m, n = L.shape             # pomocí vhodné funkce numpy uložte do proměnných m, n počet řádků a sloupců L
    x = np.zeros(n)                # vytvořte vektor samých nul délky n

    x[0] = b[0] / L[0, 0]   # první prvek řešení

    for i in range(1, m):

        # nejdříve pomocí vnořeného for cyklu určíme hodnotu sumy z kroku 2:
        suma = 0
        for j in range(i):                           # doplňte vhodně meze smyčky
            suma += L[i, j] * x[j]                        # doplňte

        # odečteme sumu od prvku pravé strany a vydělíme diagonálním prvkem
        x[i] = (b[i] - suma)/L[i, i]                             # doplňte

    return x


In [8]:
# Otestujte pomocí L, b
x = fsubst(L, b)
print(L)
print(b)
print(x)

[[0.314 0.    0.    0.   ]
 [0.301 0.53  0.    0.   ]
 [0.965 0.852 0.935 0.   ]
 [0.278 0.181 0.186 0.508]]
[0.206 0.219 0.8   0.232]
[0.657 0.041 0.14  0.032]


In [9]:
# ÚKOL: určete řešení pomocí funkce solve z numpy a porovnejte s vašim řešením
x_numpy = np.linalg.solve(L, b)
print(x_numpy)

[0.657 0.041 0.14  0.032]


### Zpětná substituce

K vyřešení soustavy s horní trojúhelníkovou maticí $\mathsf{U}\mathbf{x} = \mathbf{b}$ potřebujeme implementovat také algoritmus zpětné substituce. Ten je podobný dopředné substituci, postupuje však od posledního řádku:

1) Určíme poslední prvek vektoru řešení 
$$x_m = b_m / U_{m,m}$$
2) Určíme zbývající prvky
$$\forall i \in \{m-1, m-2, \ldots, 1\}: x_i = (b_i - \sum_{j=i+1}^{m}U_{i, j}x_j) / U_{i, i}$$

Implementujte v následující buňce vaši verzi zpětné substituce.

**Poznámka** Při implementaci se vám bude hodit vědět, že funkce range umožňuje iterovat i "pozpátku". Vyzkoušejte si v následující buňce vytisknout přes jaká *i* iterují tyto `for` cykly:
- `for i in range(6, 0, -1)`
- `for i in range(6, -1, -1)`
- `for i in range(6, 0, -2)`
- `for i in range(0, -6, -1)`

In [10]:
# ÚKOL: Vytiskněte proměnnou i z příkladů v předchozí buňce.
for i in range(6, 0, -1):
    print(i)
print()
for i in range(6, -1, -1):
    print(i)
print()
for i in range(6, 0, -2):
    print(i)
print()
for i in range(0, -6, -1):
    print(i)

6
5
4
3
2
1

6
5
4
3
2
1
0

6
4
2

0
-1
-2
-3
-4
-5


In [11]:
# ÚKOL: Implementujte algoritmus zpětné substituce.

def b_subst(U, b):
    m, n = U.shape
    x = np.zeros(n)

    x[-1] = b[-1] / U[-1, -1]

    for i in range(m-1, -1, -1):
        suma = 0
        for j in range(m-1, i, -1):
            suma += U[i, j] * x[j]
        
        x[i] = (b[i] - suma)/U[i, i]
    
    return x

In [12]:
# ÚKOL: ověřte pomocí numpy, že vaše funkce vrací správný výsledek

print(U)
print(b)
x2 = b_subst(U, b)
x2_numpy = np.linalg.solve(U, b) # doplňte
print(x2)
print(x2_numpy)

[[0.314 0.622 0.068 0.201]
 [0.    0.53  0.691 0.695]
 [0.    0.    0.935 0.499]
 [0.    0.    0.    0.508]]
[0.206 0.219 0.8   0.232]
[ 2.179 -0.984  0.612  0.457]
[ 2.179 -0.984  0.612  0.457]


# Domácí úkol č. 1

Algoritmus LU rozkladu lze popsat např. následujícím pseudokódem.

```
U = A, L = I
for k = 1 to m-1 do
    for j = k+1 to m do
        L(j, k) = U(j, k)/U(k, k)
        U(j, k:m) = U(j, k:m) - L(j, k)U(k, k:m)
    end for
end for
```

**Pozor**: značení `k:m` zde znamená sloupce od `k` po `m` *včetně*.

Vašim úkolem je:
1) Na základě uvedeného pseudokódu implementovat LU rozklad bez pivotizace do metody `L, U = my_lu(A)`. Metoda tedy bude mít na vstupu čtvercovou matici A a na výstupu dolní trojúhelníkovou matici L a horní trojúhelníkovou matici U takové, že `A = LU`.
2) Sestavit náhodnou matici A o rozměrech 5x5 a pomocí ní ověřit, že vaše metoda funguje správně (`A - L @ U` by mělo vrátit matici s nulovými či velmi malými prvky).
3) Sestavit náhodný vektor pravé strany `b` o délce 5. 
4) Pomocí vaší metody `my_lu` a na cvičení implementovaných metod `f_subst` a `b_subst` vyřešit soustavu `Ax = b`.
5) Ověřit pomocí metody `np.linalg.solve`, že jste nalezli správné řešení. 

In [30]:
# Doplňte podle bodu 1)

def my_lu(A):
    m = A.shape[0]
    U = np.copy(A)
    L = np.eye(m)

    for k in range(m-1):
        for j in range(k+1, m):
            L[j, k] = U[j, k] / U[k, k]
            U[j, k:m+1] = U[j, k:m+1] - L[j, k] * U[k, k:m+1]
    
    return (L, U)

In [45]:
# Doplňte podle bodu 2)
A = np.random.rand(5, 5)
L, U = my_lu(A)

print(A)
print(L)
print(U)
print(A - L @ U)

[[0.757 0.797 0.256 0.172 0.952]
 [0.498 0.415 0.621 0.839 0.296]
 [0.349 0.01  0.886 0.744 0.87 ]
 [0.658 0.51  0.49  0.559 0.616]
 [0.696 0.36  0.076 0.743 0.146]]
[[1.    0.    0.    0.    0.   ]
 [0.658 1.    0.    0.    0.   ]
 [0.462 3.263 1.    0.    0.   ]
 [0.869 1.667 0.687 1.    0.   ]
 [0.919 3.396 2.392 5.934 1.   ]]
[[ 7.566e-01  7.967e-01  2.563e-01  1.724e-01  9.522e-01]
 [-5.551e-17 -1.098e-01  4.528e-01  7.254e-01 -3.312e-01]
 [ 0.000e+00  0.000e+00 -7.095e-01 -1.702e+00  1.511e+00]
 [ 0.000e+00  0.000e+00 -5.551e-17  3.694e-01 -6.979e-01]
 [ 0.000e+00  0.000e+00  2.220e-16  0.000e+00  9.234e-01]]
[[ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 1.665e-16  1.388e-17 -2.220e-16 -1.110e-16 -1.110e-16]
 [ 1.110e-16  0.000e+00  1.110e-16 -1.110e-16 -1.110e-16]
 [ 2.220e-16  0.000e+00  4.580e-16 -3.331e-16  1.110e-16]]


In [46]:
# Doplňte podle bodu 3)
b = np.random.rand(5)
print(b)

[0.188 0.785 0.272 0.198 0.132]


In [47]:
# Doplňte podle bodu 4)
y = fsubst(L, b)
x = b_subst(U, y)

print(x)

[-3.594  2.536 -0.973  2.261  0.784]


In [48]:
# Doplňte podle bodu 5)
x2 = np.linalg.solve(A, b)
print(x2)

[-3.594  2.536 -0.973  2.261  0.784]
